In [1]:
# read data
import xlrd
names, coords, supply, demand = [], [], [], []
# coord = [(lat, long)..]
# demand = [..]
# supply = [..]
# data_file = open('Food_allocation.xlsx', 'r')
wb = xlrd.open_workbook('Food_allocation.xlsx')
sheet = wb.sheet_by_index(0)
for i in range(1, sheet.nrows):
    names.append(sheet.cell_value(i, 0))
    supply.append(sheet.cell_value(i, 1))
    demand.append(sheet.cell_value(i, 2))
    coords.append((sheet.cell_value(i, 3), sheet.cell_value(i, 4)))

def get_target_values(supply, demand, node=None):
    adjusted_supply = [s - min(d, s) for s, d in zip(supply, demand)]
    adjusted_demand = [d - min(d, s) for s, d in zip(supply, demand)]

    supply_sum = float(sum(adjusted_supply))
    demand_sum = float(sum(adjusted_demand))
    # short, target = sorted(supply_tup, demand_tup, key=lembda l: l[1])
    supply_ratio = min(demand_sum/supply_sum, 1.0)
    demand_ratio = min((supply_sum - adjusted_demand[node])/(demand_sum - adjusted_demand[node]), 1.0)\
                    if node and adjusted_demand[node] <= supply_sum else min(supply_sum/demand_sum, 1.0)
    sources = [s * supply_ratio for s in adjusted_supply]
    targets = [d * demand_ratio for d in adjusted_demand]
    if node: targets[node] = adjusted_demand[node]
    
    return sources, targets


In [8]:
# prepare distance_matrix
sources, targets = get_target_values(supply, demand, 2)
# distance_matrix = {(i, j): dist, .. }
distance_matrix = [[1 for s in sources] for t in targets]
print (sources)
print (targets)

[0.0, 4500.0, 0.0, 0.0, 1500.0, 0.0, 0.0, 0.0, 4400.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1100.0, 4700.0, 6000.0, 0.0, 9700.0]
[4658.469945355191, 0.0, 1300.0, 8105.737704918033, 0.0, 1677.049180327869, 465.8469945355191, 1211.2021857923498, 0.0, 0.0, 838.5245901639345, 2515.5737704918033, 1956.5573770491803, 7267.213114754099, 559.016393442623, 0.0, 0.0, 0.0, 4844.808743169399, 0.0]


In [3]:
# Contraints
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
from itertools import product

#model = cp_model.CpModel()

solver = pywraplp.Solver('ResourceAllocation', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
# sources, targets = get_target_values(supply, demand)

source_upper_bound = max(sources)
target_upper_bound = max(targets)

transfers = {}
bin_transfers = {}

for i, j in product(range(len(sources)), range(len(targets))):
    transfers[(i, j)] = solver.NumVar(0, int(source_upper_bound) + 1, 'transfer_%i_to_%i' % (i, j))
    
    # bin_transfers[(i, j)] = model.NewIntVar(0, 1, 'bin_transfer_%i_to_%i' % (i,j))
    # model.Add(bin_transfers[(i, j)] <= transfers[(i, j)])
    # model.Add(bin_transfers[(i, j)] * int(source_upper_bound) >= transfers[(i, j)])
cons1 = {}
cons2 = {}
for i in range(len(sources)):
    cons1[i] = solver.Constraint(sources[i], sources[i])
    cons2[i] = solver.Constraint(targets[i], targets[i])
    for j in range(len(targets)):
        cons1[i].SetCoefficient(transfers[(i, j)], 1)
        cons2[i].SetCoefficient(transfers[(j, i)], 1)
    # model.Add(sources[i] >= sum(transfers[(i, j)] for j in range(len(sources))))
    # model.Add(targets[i] <= sum(transfers[(j, i)] for j in range(len(sources))))

In [4]:
# objective function
objective = solver.Objective()
# objective.SetCoefficient(x, 3)
# objective.SetCoefficient(y, 4)
# objective.SetMaximization()
for i in range(len(sources)):
    for j in range(len(targets)):
        objective.SetCoefficient(transfers[(i, j)], distance_matrix[i][j])
# model.Minimize(sum(transfers[(i, j)] * distance_matrix[i][j] for i in range(len(sources)) for j in range(len(targets)) ))

In [5]:
# optimizer
# solver = cp_model.CpSolver()
# solver.Solve(model)

solver.Solve()

result = {}
for i, j in product(range(len(sources)), range(len(targets))):
    result[(i, j)] = transfers[(i, j)].solution_value()

In [6]:
# postprocess
def print_result(results):
    str_res = ''''''
    for i in range(len(sources)):
        line = ''
        for j in range(len(sources)):
            line += ' ' + str(result[(i, j)])
        str_res += '\n' + line
    print (str_res)
print_result(result)

[print (str(i+1) + ' to ' + str(j+1) + ' tranfer '+ str(result[(i, j)])) for i, j in product(range(len(sources)), range(len(targets))) if result[(i, j)] > 0 ]


 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 4500.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 288.79781420765033 0.0 0.0 0.0 0.0 0.0 0.0 1211.2021857923498 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 655.191256830601 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3744.808743169399 0.0
 1495.9016393442625 0.0 143.71584699453575 0.0 0.0 1021.8579234972678 0.0 0.0 0.0 0.0 838.5245901639345 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [5]:
'transfer_%i_to_%i' % (1, 2)

'transfer_1_to_2'